In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from os.path import expanduser, join
from re import findall
from pandas import DataFrame, to_datetime, merge

In [9]:
home = expanduser('~')
mypath = f'{home}/Downloads/BigData/'
deromalized_path = join(mypath, 'Datastax_denormalized/').replace('\\', '/')

## Connect to DataStax

In [10]:
cloud_config= {'secure_connect_bundle': mypath+'secure-connect-bigdataproject2022.zip'}

with open(mypath+'GeneratedToken.csv', 'r', encoding='utf-8') as file:
    try:
        content = file.readlines()        
    except FileNotFoundError:
        raise('File not found')

tokens = findall(r'"(.*?)"', content[1])
client_id = tokens[0]
client_secret = tokens[1]

In [11]:
keyspace = 'big_data'
auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect(keyspace)

## Q1

## Retrieve data for Q1

In [12]:
def select_from_popular_movies_by_date(start_date, end_date):
    sd = to_datetime(start_date)
    ed = to_datetime(end_date)
    if sd.year == ed.year:
        months = ','.join([str(i) for i in range(sd.month, ed.month+1)])
        Q1 = f"SELECT movieid, title, genres, rating FROM popular_movies_by_date WHERE year={sd.year} AND month IN({months}) AND day>={sd.day} AND day<={ed.day};"
        query_results = session.execute(Q1)
        
        movies = DataFrame([(row.movieid, row.title, row.genres, row.rating) for row in query_results], columns=['movieid', 'title', 'genres', 'rating'])
        u_movies = DataFrame(movies.groupby(by='movieid')['rating'].mean())
        u_movies.reset_index(inplace=True)
        u_movies.sort_values(by='rating', ascending=False, inplace=True)
        movies = merge(left=u_movies, right=movies.loc[:, 'movieid':'genres'], left_on='movieid', right_on='movieid')
        
        for index, row in movies.iterrows():
            print(row.title, row.rating)
            if index == 29:
                break
    else:
        print("Invalid date range!")

In [17]:
select_from_popular_movies_by_date('2015-01-01', '2015-01-15')

What Ever Happened to Baby Jane? 5.0
Shrek 4.199999809265137
Air America 3.0


## Q2

## Retrieve data for Q2

In [20]:
def select_from_movies_by_keyword(keyword='toy', limit=5):
    Q2 = f"SELECT title, rating, genres FROM movies_by_keyword WHERE keyword='{keyword.lower()}' LIMIT {limit};"
    rows = session.execute(Q2)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.rating:.1f} Genres: {row.genres}")

In [22]:
select_from_movies_by_keyword(keyword='star')

Title: Star Wars: Episode IV - A New Hope Rating: 4.2 Genres: Action|Adventure|Sci-Fi
Title: Star Wars: Episode V - The Empire Strikes Back Rating: 4.2 Genres: Action|Adventure|Sci-Fi
Title: Lucky Star Rating: 4.2 Genres: Drama|Romance
Title: Lone Star Rating: 4.1 Genres: Drama|Mystery|Western
Title: The Cloud-Capped Star Rating: 4.1 Genres: Drama


# Q3

## Retrieve data for Q2

In [23]:
def select_from_movies_by_genre(genre, limit=10):
    Q3 = f"SELECT title, avg_rating, release_date FROM movies_by_genre WHERE genre='{genre}' ORDER BY release_date ASC LIMIT {limit};"
    rows = session.execute(Q3)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.avg_rating:.1f} Release Date: {row.release_date}")
    

In [24]:
# insert_into_movies_by_genre(movies_ratings, 1)
select_from_movies_by_genre('Adventure')

Title: A Trip to the Moon Rating: 3.7 Release Date: 1902
Title: The Last Days of Pompeii Rating: 2.5 Release Date: 1913
Title: Judith of Bethulia Rating: 3.3 Release Date: 1914
Title: Cabiria Rating: 3.1 Release Date: 1914
Title: Vampires, Les Rating: 3.4 Release Date: 1915
Title: Judex Rating: 3.5 Release Date: 1916
Title: 20,000 Leagues Under the Sea Rating: 3.3 Release Date: 1916
Title: Male and Female Rating: 2.9 Release Date: 1919
Title: The Spiders Part 1: The Golden Lake Rating: 2.8 Release Date: 1919
Title: The Last of the Mohicans Rating: 3.4 Release Date: 1920


## Q4

## Retrieve data for Q4

In [25]:
def select_from_movies_by_title(session, title):
    Q4 = f"SELECT title, genres, tag, avg_rating AS avg FROM movie_info_by_title WHERE title='{title}' ORDER BY tag_count LIMIT 5;"
    rows = session.execute(Q4)
    print(f"Title: {rows[0].title} \nRating: {rows[0].avg:.1f} \nGenres: {rows[0].genres}")
    print(f"Most popular tags: ")
    for row in rows:
        print(row.tag)

In [26]:
select_from_movies_by_title(session, 'Jumanji')

Title: Jumanji 
Rating: 3.2 
Genres: Adventure|Children|Fantasy
Most popular tags: 
time travel
time
see alsoZathura
scary
not for kids


## Q5

## Retrieve data for Q5

In [27]:
def select_from_movies_by_tag(tag):
    Q5 = f"SELECT title, avg_rating as avg, genres FROM movies_by_tag WHERE tag='{tag}' ORDER BY avg_rating DESC LIMIT 10;"
    rows = session.execute(Q5)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.avg:.1f}")

In [28]:
select_from_movies_by_tag('Comedy')

Title: Some Like It Hot Rating: 4.1
Title: Forrest Gump Rating: 4.0
Title: Indiana Jones and the Last Crusade Rating: 4.0
Title: WALL·E Rating: 4.0
Title: Toy Story 3 Rating: 4.0
Title: Femme Nikita, La Rating: 3.9
Title: Groundhog Day Rating: 3.9
Title: Back to the Future Rating: 3.9
Title: Monsters, Inc. Rating: 3.9
Title: Finding Nemo Rating: 3.9
